In [112]:
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit

def pacejka_model(alpha, B, C, D, E, F):
    return D * np.sin(C * np.arctan(B * alpha - E * (B * alpha - np.arctan(B * alpha))) + F)

def process_tire_data(file_path):
    metric_data = pd.read_csv(file_path)

    for col in metric_data.columns:
        metric_data.rename(columns={col: f"{col} {metric_data[col][0]}"}, inplace=True)

    metric_data = metric_data.drop(0)
    metric_data.reset_index(drop=True, inplace=True)
    metric_data = metric_data.apply(pd.to_numeric, errors='coerce')
    metric_data['FY N'] = -metric_data['FY N']
    metric_data = metric_data.dropna()
    metric_data.reset_index(drop=True, inplace=True)

    # Adaptive binning based on quantiles for 'FZ N'
    metric_data['Load Bin'] = pd.qcut(metric_data['FZ N'], q=1000, labels=False)  # 10 bins as example

    binned_data = []
    for _, group in metric_data.groupby('Load Bin'):
        mean_SA = group['SA deg'].mean()
        mean_FY = group['FY N'].mean()
        # Assuming pressure and camber are still relevant, use their mean or median as needed
        mean_pressure = group['P kPa'].mean()
        mean_camber = group['IA deg'].mean()
        binned_data.append([mean_pressure, mean_camber, _, mean_SA, mean_FY])

    binned_df = pd.DataFrame(binned_data, columns=['Pressure', 'Camber', 'Load Bin', 'Mean SA', 'Mean FY'])
    initial_guess = [8, 1.3, 3500, -1, 0]
    #initial_guess = [10, 1.5, max(metric_data['FY N']), 0.1, 1]
    bounds = (-99999999, np.inf)

    if not binned_df.empty and len(binned_df) > len(initial_guess):
        params_optimal, _ = curve_fit(pacejka_model, binned_df['Mean SA'], binned_df['Mean FY'],
                                      p0=initial_guess, bounds=bounds, maxfev=150000)
    else:
        raise ValueError("Not enough data to fit the model or binned data is empty.")

    return params_optimal, binned_df


In [113]:
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
import plotly.graph_objects as go
import os

# Assuming the function process_tire_data is already defined and returns fitted params and binned data

def fitted_curve(alpha_range, B, C, D, E, F):
    return pacejka_model(alpha_range, B, C, D, E, F)

# List of file paths for different datasets
file_paths = [
    #'B1320run50.csv', 'B1320run51.csv','B1320run52.csv','B1320run53.csv',
    'B1320run54.csv','B1320run124.csv',
    #'B1320run125.csv','B1320run126.csv'#,'B1320run127.csv','B1320run128.csv','B1320run129.csv',
    # ... add as many dataset file paths as needed
]

# Dictionary to store results for each dataset
tire_data_results = {}
coefficients_df = pd.DataFrame(columns=['File Name', 'B', 'C', 'D', 'E', 'F'])


# Process each dataset file and store the results
for file_path in file_paths:
    params_optimal, binned_df = process_tire_data(file_path)
    tire_data_results[file_path] = {'params': params_optimal, 'binned_df': binned_df}

    # Extract just the run number from the file path for the DataFrame
    run_name = os.path.splitext(os.path.basename(file_path))[0]

    # Append the coefficients to the DataFrame
    coefficients_df = coefficients_df.append({
        'File Name': run_name,
        'B': params_optimal[0],
        'C': params_optimal[1],
        'D': params_optimal[2],
        'E': params_optimal[3],
        'F': params_optimal[4]
    }, ignore_index=True)
# Plot all the fitted curves using Plotly
# Initialize Plotly figure
fig = go.Figure()

# Assuming tire_data_results is populated as before
for file_path, data in tire_data_results.items():
    # Generate a range of slip angles for plotting
    alpha_range = np.linspace(data['binned_df']['Mean SA'].min(), data['binned_df']['Mean SA'].max(), num=500)

    # Calculate the fitted curve over the range of alpha
    fy_fitted = fitted_curve(alpha_range, *data['params'])

    # Extract the run number from the file path for the legend title
    run_name = os.path.splitext(os.path.basename(file_path))[0]  # This will extract 'run50' from 'run50.csv'

    # Add the fitted curve to the plot with the run name as the legend title
    fig.add_trace(go.Scatter(x=alpha_range, y=fy_fitted, mode='lines', name=run_name))

# Update plot layout
fig.update_layout(
    title='Comparison of Fitted Tire Curves',
    xaxis_title='Slip Angle (SA deg)',
    yaxis_title='Lateral Force (FY N)',
    legend_title='Dataset'
)

# Show plot
#fig.show()
coefficients_df

<ipython-input-113-94b8311b4088>:34: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-113-94b8311b4088>:34: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



,File Name,B,C,D,E,F
0,B1320run54,323.401082,-15.099382,-833.127065,-13.383496,-10.189344
1,B1320run124,376.950435,-16.736051,-610.760445,-2.582083,-0.024802


In [114]:
import os
import plotly.graph_objects as go
import plotly.express as px

# Initialize Plotly figure for the raw data comparison
fig_raw = go.Figure()

# Assuming tire_data_results is populated as before
for file_path, data in tire_data_results.items():
    # Extract the run number from the file path for the legend title
    run_name = os.path.splitext(os.path.basename(file_path))[0]  # This will extract 'run50' from 'run50.csv'

    # Add the raw data to the plot with the run name as the legend title
    fig_raw.add_trace(go.Scatter(x=data['binned_df']['Mean SA'], y=data['binned_df']['Mean FY'], mode='markers', name=run_name))

# Update plot layout for the raw data comparison
fig_raw.update_layout(
    title='Comparison of Raw Tire Data',
    xaxis_title='Slip Angle (SA deg)',
    yaxis_title='Lateral Force (FY N)',
    legend_title='Dataset'
)

# Show the raw data comparison plot
fig_raw.show()

# Initialize Plotly figure for the fitted curves comparison
fig_fitted = go.Figure()

# Use the same colors for each run in both plots
color_palette = px.colors.qualitative.Plotly

# Plot raw data and fitted curves
for index, (file_path, data) in enumerate(tire_data_results.items()):
    # Choose color
    color = color_palette[index % len(color_palette)]

    # Extract the run number from the file path for the legend title
    run_name = os.path.splitext(os.path.basename(file_path))[0]

    # Generate a range of slip angles for plotting
    alpha_range = np.linspace(data['binned_df']['Mean SA'].min(), data['binned_df']['Mean SA'].max(), num=500)

    # Calculate the fitted curve over the range of alpha
    fy_fitted = fitted_curve(alpha_range, *data['params'])

    # Add the raw data to the plot
    #fig_fitted.add_trace(go.Scatter(x=data['binned_df']['Mean SA'], y=data['binned_df']['Mean FY'], mode='markers', name=run_name, marker_color=color))

    # Add the fitted curve to the plot
    fig_fitted.add_trace(go.Scatter(x=alpha_range, y=fy_fitted, mode='lines', name=f'Fitted {run_name}', line=dict(color=color)))

# Update plot layout for the fitted curves comparison
fig_fitted.update_layout(
    title='Comparison of Fitted Tire Curves',
    xaxis_title='Slip Angle (SA deg)',
    yaxis_title='Lateral Force (FY N)',
    legend_title='Dataset'
)

fig_raw.update_yaxes(range=[-4000, 4000], dtick=100)
fig_fitted.update_yaxes(range=[-4000, 4000], dtick=250)


# Show the fitted curves comparison plot
fig_fitted.show()


In [33]:
coefficients_df

,File Name,B,C,D,E,F
0,B1320run50,0.217053,0.333634,8898.515534,0.517105,6.235110e-07


In [126]:
test_data = pd.read_csv("B1320run54.csv")
test_data = test_data.apply(pd.to_numeric, errors='coerce')
test_data = test_data.dropna()
test_data.columns

Index(['ET', 'V', 'N', 'SA', 'IA', 'RL', 'RE', 'P', 'FX', 'FY', 'FZ', 'MX',
       'MZ', 'NFX', 'NFY', 'RST', 'TSTI', 'TSTC', 'TSTO', 'AMBTMP', 'SR'],
      dtype='object')

In [115]:
import os
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from sklearn.metrics import r2_score
# Initialize Plotly figure for overlaying fitted curves and raw data
fig_overlay = go.Figure()

# Use the same color palette for consistency
color_palette = px.colors.qualitative.Plotly

# Overlay raw data and fitted curves with R^2 calculation
for index, (file_path, data) in enumerate(tire_data_results.items()):
    color = color_palette[index % len(color_palette)]
    run_name = os.path.splitext(os.path.basename(file_path))[0]

    # Raw data
    fig_overlay.add_trace(go.Scatter(x=data['binned_df']['Mean SA'], y=data['binned_df']['Mean FY'],
                                     mode='markers', name=f'Raw {run_name}', marker_color=color))

    # Generate fitted curve
    alpha_range = np.linspace(data['binned_df']['Mean SA'].min(), data['binned_df']['Mean SA'].max(), num=500)
    fy_fitted = fitted_curve(alpha_range, *data['params'])

    # Add fitted curve to plot
    fig_overlay.add_trace(go.Scatter(x=alpha_range, y=fy_fitted, mode='lines',
                                     name=f'Fitted {run_name}', line=dict(color=color)))

    # Calculate R^2 and print or add to the plot title/legend
    r_squared = r2_score(data['binned_df']['Mean FY'], fitted_curve(data['binned_df']['Mean SA'], *data['params']))
    print(f"{run_name} R^2: {r_squared}")
    # Optionally, you can add R^2 value to the legend by appending it to the name of the fitted curve trace

# Update plot layout
fig_overlay.update_layout(
    title='Overlay of Raw Data and Fitted Curves with R^2 Values',
    xaxis_title='Slip Angle (SA deg)',
    yaxis_title='Lateral Force (FY N)',
    legend_title='Dataset'
)

# Adjust the y-axis range if needed
fig_overlay.update_yaxes(range=[-4000, 4000], dtick=250)

# Show the plot
fig_overlay.show()

B1320run54 R^2: 0.5823348184003958
B1320run124 R^2: 0.5713227044571929


In [116]:
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
import plotly.graph_objects as go

# Define the path to the CSV file
data_path = 'B1320run50.csv'

# Load the data, skipping the first row which contains units
data = pd.read_csv(data_path, skiprows=1)

# Convert relevant columns to numeric types, assuming 'deg' is slip angle and 'N.1' is lateral force
data['SA'] = pd.to_numeric(data['deg'], errors='coerce')
data['FY'] = pd.to_numeric(data['N.1'], errors='coerce')

# Clean the data by dropping rows with NaN values in the columns of interest
cleaned_data = data[['SA', 'FY']].dropna()

# Define the simplified Pacejka model function
def pacejka_model(alpha, B, C, D, E, F):
    return D * np.sin(C * np.arctan(B * alpha - E * (B * alpha - np.arctan(B * alpha))) + F)

# Prepare the data for curve fitting
x_data = cleaned_data['SA']
y_data = cleaned_data['FY']

# Apply curve fitting to find the optimal coefficients
# The Levenberg-Marquardt algorithm is used by default in curve_fit
initial_guess = [1, 1, 1, 1, 1]
optimal_parameters, covariance = curve_fit(pacejka_model, x_data, y_data, p0=initial_guess)

# Generate model predictions over the range of slip angles for visualization
x_model = np.linspace(x_data.min(), x_data.max(), 500)
y_model = pacejka_model(x_model, *optimal_parameters)

# Calculate R^2 accuracy
y_pred = pacejka_model(x_data, *optimal_parameters)
ss_res = np.sum((y_data - y_pred) ** 2)
ss_tot = np.sum((y_data - np.mean(y_data)) ** 2)
r_squared = 1 - (ss_res / ss_tot)

# Creating the visualization with Plotly
fig = go.Figure()

# Actual data points
fig.add_trace(go.Scatter(x=x_data, y=y_data, mode='markers', name='Actual Data'))

# Fitted model curve
fig.add_trace(go.Scatter(x=x_model, y=y_model, mode='lines', name='Fitted Model'))

# Enhancing the plot
fig.update_layout(title=f"Pacejka Model Fitting with R²: {r_squared:.3f}",
                  xaxis_title="Slip Angle (degrees)",
                  yaxis_title="Lateral Force (N)",
                  legend_title="Legend")

# Display the plot
fig.show()

# Print the optimal parameters and R^2 value
print(f"Optimal Parameters:\nB: {optimal_parameters[0]}\nC: {optimal_parameters[1]}\nD: {optimal_parameters[2]}\nE: {optimal_parameters[3]}\nF: {optimal_parameters[4]}")
print(f"R² Accuracy: {r_squared:.3f}")


Optimal Parameters:
B: -79.47179513495368
C: -16.953510400325182
D: -1660.6323325577098
E: 11.621462476738202
F: 28.282553598840348
R² Accuracy: 0.705


In [148]:


# Define the simplified Pacejka model function
def pacejka_model(alpha, B, C, D, E, F):
    return D * np.sin(C * np.arctan(B * alpha - E * (B * alpha - np.arctan(B * alpha))) + F)

import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Define the path to the CSV file
data_path = 'B1320run50.csv'

# Load the data, skipping the first row which contains units
data = pd.read_csv(data_path, skiprows=1)

# Convert relevant columns to numeric types, assuming 'deg' is slip angle, 'N.1' is lateral force, and 'N.2' is vertical load
data['SA'] = pd.to_numeric(data['deg'], errors='coerce')
data['FY'] = pd.to_numeric(data['N.1'], errors='coerce')
data['FZ'] = pd.to_numeric(data['N.2'], errors='coerce')

data['FY'] = data['FY'] * -1

# Define vertical load ranges for splitting the data
load_ranges = [(-1600, -1200), (-1200, -800), (-800, -400), (-400, 0)]

# Create a subplot figure with the number of load ranges
fig = make_subplots(rows=len(load_ranges), cols=1, subplot_titles=[f'Load Range {lr}' for lr in load_ranges])

# Process each load range
for i, (lower_bound, upper_bound) in enumerate(load_ranges, start=1):
    # Filter the data based on the current load range
    load_data = data[(data['FZ'] >= lower_bound) & (data['FZ'] < upper_bound)]

    # Prepare the data for curve fitting
    x_data = load_data['SA']
    y_data = load_data['FY']

    # Apply curve fitting to find the optimal coefficients for each load range
    initial_guess = [0.5, 1.2, max(load_data['FY']), 1, 0]
    optimal_parameters, covariance = curve_fit(pacejka_model, x_data, y_data, p0=initial_guess, maxfev=15000)

    # Generate model predictions over the range of slip angles for visualization
    x_model = np.linspace(x_data.min(), x_data.max(), 500)
    y_model = pacejka_model(x_model, *optimal_parameters)

    # Calculate R^2 accuracy for each fitted curve
    y_pred = pacejka_model(x_data, *optimal_parameters)
    ss_res = np.sum((y_data - y_pred) ** 2)
    ss_tot = np.sum((y_data - np.mean(y_data)) ** 2)
    r_squared = 1 - (ss_res / ss_tot)

    # Add the scatter plot for actual data and the line plot for the fitted curve to the subplot
    fig.add_trace(go.Scatter(x=x_data, y=y_data, mode='markers', name=f'Actual Data {i}'), row=i, col=1)
    fig.add_trace(go.Scatter(x=x_model, y=y_model, mode='lines', name=f'Fitted Model {i}'), row=i, col=1)

    # Print the optimal parameters and R^2 value for the current load range
    print(f"Load Range {lower_bound} to {upper_bound}")
    print(f"Optimal Parameters:\nB: {optimal_parameters[0]}\nC: {optimal_parameters[1]}\nD: {optimal_parameters[2]}\nE: {optimal_parameters[3]}\nF: {optimal_parameters[4]}")
    print(f"R² Accuracy: {r_squared:.3f}\n")

# Enhancing the plot
fig.update_layout(height=1000, title_text="Pacejka Model Fitting by Load Range")

# Display the plot
fig.show()



Load Range -1600 to -1200
Optimal Parameters:
B: 0.09765732755510383
C: 2.9148553697148567
D: 3567.0038815912626
E: 1.4326014148841144
F: -0.060765353229814606
R² Accuracy: 0.994

Load Range -1200 to -800
Optimal Parameters:
B: 0.11861586163964191
C: 3.0392333734554513
D: 2705.6220245234613
E: 2.095242674686501
F: -0.011134053186851374
R² Accuracy: 0.988

Load Range -800 to -400
Optimal Parameters:
B: 0.25775309785023737
C: 1.303663560926847
D: 1715.1772477250831
E: 1.212379867078117
F: -0.0052201030826252905
R² Accuracy: 0.963

Load Range -400 to 0
Optimal Parameters:
B: 0.35095422674405014
C: 1.359943546026468
D: 590.7026795915351
E: 0.41202332474683084
F: 0.02253894424733349
R² Accuracy: 0.993



In [124]:
# Define the path to the CSV file
data_path = 'B1320run50.csv'

# Load the data
data = pd.read_csv(data_path, skiprows=1)

In [147]:
test_data = pd.read_csv("B1320run54.csv")
test_data = test_data.apply(pd.to_numeric, errors='coerce')
test_data = test_data.dropna()

# Assuming 'data' is your DataFrame and it has been cleaned and numeric columns have been created

import plotly.express as px

# Let's generate a scatter plot of 'FZ' vs 'SA' colored by 'IA' to help identify where to split the data
fig = px.scatter(test_data, x='SA', y='FZ', color='IA',
                 labels={'SA': 'Slip Angle', 'FZ': 'Vertical Load', 'IA': 'Inclination Angle'},
                 title='Vertical Load vs. Slip Angle Colored by Inclination Angle')

fig.show()

